# Response

> SSE response builders for complex UI updates

In [ ]:
#| default_exp core.response

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Any, Optional, Callable, Union, Tuple
from dataclasses import dataclass, field
from fasthtml.common import Div, FT, sse_message
from cjm_fasthtml_sse.core.streaming import OOBStreamBuilder

## Core Classes

In [ ]:
#| export
@dataclass
class UpdateRule:
    """Rule for conditional element updates"""
    condition: Callable  # Function that returns True if rule should apply
    builder: Callable  # Function that builds the element(s)
    target_id: Optional[str] = None  # Target element ID for OOB swap
    swap_mode: str = "innerHTML"  # Swap mode
    priority: int = 0  # Higher priority rules are evaluated first

In [ ]:
#| export
class SSEResponseBuilder:
    """Builder for complex SSE responses with conditional updates"""
    
    def __init__(
        self,
        debug: bool = False  # Enable debug logging
    ):
        """Initialize the response builder."""
        self.debug = debug
        self.rules: List[UpdateRule] = []
        self.always_include: List[Callable] = []
        self.context: Dict[str, Any] = {}
    
    def add_rule(
        self,
        condition: Callable,  # Condition function
        builder: Callable,  # Element builder function
        target_id: Optional[str] = None,  # Target ID for OOB
        swap_mode: str = "innerHTML",  # Swap mode
        priority: int = 0  # Rule priority
    ) -> 'SSEResponseBuilder':  # Self for chaining
        """Add a conditional update rule."""
        rule = UpdateRule(
            condition=condition,
            builder=builder,
            target_id=target_id,
            swap_mode=swap_mode,
            priority=priority
        )
        self.rules.append(rule)
        # Sort by priority (highest first)
        self.rules.sort(key=lambda r: r.priority, reverse=True)
        return self
    
    def add_always(
        self,
        builder: Callable  # Element builder that always runs
    ) -> 'SSEResponseBuilder':  # Self for chaining
        """Add a builder that always runs."""
        self.always_include.append(builder)
        return self
    
    def set_context(
        self,
        **kwargs  # Context variables
    ) -> 'SSEResponseBuilder':  # Self for chaining
        """Set context variables for builders."""
        self.context.update(kwargs)
        return self
    
    def build(
        self,
        **kwargs  # Additional context for this build
    ) -> FT:  # Built response
        """Build the response based on rules and context."""
        # Merge contexts
        build_context = {**self.context, **kwargs}
        
        # Use OOBStreamBuilder for clean element construction
        builder = OOBStreamBuilder()
        
        # Always include elements
        for always_builder in self.always_include:
            elements = always_builder(**build_context)
            if elements:
                if isinstance(elements, list):
                    for elem in elements:
                        builder.add_element(elem)
                else:
                    builder.add_element(elements)
        
        # Apply conditional rules
        for rule in self.rules:
            if rule.condition(**build_context):
                elements = rule.builder(**build_context)
                if elements:
                    if isinstance(elements, list):
                        for elem in elements:
                            builder.add_element(
                                elem,
                                target_id=rule.target_id,
                                swap_mode=rule.swap_mode
                            )
                    else:
                        builder.add_element(
                            elements,
                            target_id=rule.target_id,
                            swap_mode=rule.swap_mode
                        )
                
                if self.debug:
                    print(f"[SSEResponseBuilder] Applied rule with priority {rule.priority}")
        
        return builder.build()
    
    def clear_rules(
        self
    ) -> 'SSEResponseBuilder':  # Self for chaining
        """Clear all rules."""
        self.rules = []
        return self
    
    def clear_always(
        self
    ) -> 'SSEResponseBuilder':  # Self for chaining
        """Clear always-include builders."""
        self.always_include = []
        return self

### Tests for SSEResponseBuilder

In [ ]:
#| test
# Test SSEResponseBuilder initialization
builder = SSEResponseBuilder(debug=False)

assert builder.debug == False
assert len(builder.rules) == 0
assert len(builder.always_include) == 0
assert len(builder.context) == 0

print("✓ SSEResponseBuilder initialization tests passed")

✓ SSEResponseBuilder initialization tests passed


In [ ]:
#| test
# Test adding rules and always-include builders
from fasthtml.common import P, Span

builder = SSEResponseBuilder()

# Add always-include builder
def always_stats(**ctx):
    return P(f"Count: {ctx.get('count', 0)}")

builder.add_always(always_stats)
assert len(builder.always_include) == 1

# Add conditional rules
def is_active(**ctx):
    return ctx.get('status') == 'active'

def active_element(**ctx):
    return Span("Active", id="status")

builder.add_rule(
    condition=is_active,
    builder=active_element,
    target_id="status-container",
    priority=10
)

assert len(builder.rules) == 1
assert builder.rules[0].priority == 10

# Add another rule with lower priority
def is_error(**ctx):
    return ctx.get('error', False)

def error_element(**ctx):
    return Span("Error!", id="error")

builder.add_rule(
    condition=is_error,
    builder=error_element,
    priority=5
)

assert len(builder.rules) == 2
# Check rules are sorted by priority
assert builder.rules[0].priority == 10
assert builder.rules[1].priority == 5

print("✓ SSEResponseBuilder rule management tests passed")

✓ SSEResponseBuilder rule management tests passed


In [ ]:
#| test
# Test context and building
from fasthtml.common import P, Div

builder = SSEResponseBuilder()

# Set context
builder.set_context(user="test_user", role="admin")
assert builder.context["user"] == "test_user"
assert builder.context["role"] == "admin"

# Add builders
def user_info(**ctx):
    return P(f"User: {ctx.get('user', 'unknown')}")

def admin_panel(**ctx):
    return Div("Admin Panel", id="admin")

builder.add_always(user_info)
builder.add_rule(
    condition=lambda **ctx: ctx.get('role') == 'admin',
    builder=admin_panel
)

# Build with context
result = builder.build()
assert result is not None

# Build with override context
result2 = builder.build(role="user")
assert result2 is not None

# Clear and rebuild
builder.clear_rules().clear_always()
assert len(builder.rules) == 0
assert len(builder.always_include) == 0

print("✓ SSEResponseBuilder context and build tests passed")

✓ SSEResponseBuilder context and build tests passed


## Helper Functions

In [ ]:
#| export
def create_conditional_response(
    conditions: List[Tuple[Callable, Callable]],  # List of (condition, builder) tuples
    always_include: Optional[List[Callable]] = None,  # Builders that always run
    context: Optional[Dict[str, Any]] = None  # Initial context
) -> SSEResponseBuilder:  # Configured response builder
    """Create a response builder with predefined conditions."""
    builder = SSEResponseBuilder()
    
    # Set initial context
    if context:
        builder.set_context(**context)
    
    # Add always-include builders
    if always_include:
        for always_builder in always_include:
            builder.add_always(always_builder)
    
    # Add conditional rules
    for i, (condition, element_builder) in enumerate(conditions):
        builder.add_rule(
            condition=condition,
            builder=element_builder,
            priority=len(conditions) - i  # Higher priority for earlier rules
        )
    
    return builder

In [ ]:
#| export
def create_state_response_builder(
    state_builders: Dict[str, Callable],  # Mapping of state names to builders
    get_state_fn: Callable,  # Function to determine current state
    default_builder: Optional[Callable] = None  # Default builder if no state matches
) -> SSEResponseBuilder:  # Configured response builder
    """Create a response builder for state-based updates."""
    builder = SSEResponseBuilder()
    
    # Add rules for each state
    for state_name, state_builder in state_builders.items():
        builder.add_rule(
            condition=lambda s=state_name, **ctx: get_state_fn(**ctx) == s,
            builder=state_builder
        )
    
    # Add default if provided
    if default_builder:
        builder.add_rule(
            condition=lambda **ctx: get_state_fn(**ctx) not in state_builders,
            builder=default_builder,
            priority=-1  # Lowest priority
        )
    
    return builder

In [ ]:
#| test
# Test create_conditional_response
from fasthtml.common import P, Div

conditions = [
    (lambda **ctx: ctx.get('level') > 5, lambda **ctx: P("High level")),
    (lambda **ctx: ctx.get('level') > 0, lambda **ctx: P("Low level")),
]

always = [lambda **ctx: Div("Header")]

builder = create_conditional_response(
    conditions=conditions,
    always_include=always,
    context={"app": "test"}
)

assert len(builder.rules) == 2
assert len(builder.always_include) == 1
assert builder.context["app"] == "test"

print("✓ create_conditional_response tests passed")

✓ create_conditional_response tests passed


In [ ]:
#| test
# Test create_state_response_builder
from fasthtml.common import Span

state_builders = {
    "running": lambda **ctx: Span("Running..."),
    "complete": lambda **ctx: Span("Complete!"),
    "error": lambda **ctx: Span("Error!")
}

def get_state(**ctx):
    return ctx.get('status', 'unknown')

def default(**ctx):
    return Span("Unknown state")

builder = create_state_response_builder(
    state_builders=state_builders,
    get_state_fn=get_state,
    default_builder=default
)

assert len(builder.rules) == 4  # 3 states + 1 default

# Test building with different states
result_running = builder.build(status="running")
assert result_running is not None

result_unknown = builder.build(status="unknown")
assert result_unknown is not None

print("✓ create_state_response_builder tests passed")

✓ create_state_response_builder tests passed


## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()